In [3]:
!pip install beautifulsoup4
!pip install lxml

In [5]:
from bs4 import BeautifulSoup

In [7]:
import requests

url = "https://www2.hse.ie/services/pharmacies-flu-and-covid-vaccines/"

response = requests.get(url)
if response.status_code == 200:
    html_content = response.text
else:
    print("Failed to retrieve the webpage. Status code:", response.status_code)

soup = BeautifulSoup(html_content, 'lxml')

In [19]:
element = soup.find(class_='hse-form-group')
print(element.text)


Search the website




In [35]:
dropdown = soup.find("select", id = "service_area")

if dropdown:
    options = dropdown.find_all("option")
    for option in options:
        print(f"Text: {option.text.strip()}, Value: {option.get('value')}")

In [37]:
print(dropdown)

None


In [39]:
print(soup.prettify())

<!DOCTYPE html>
<html class="no-js" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Find a pharmacy giving flu and COVID-19 vaccines
            
            
                
                    - HSE.ie
  </title>
  <link href="https://www2.hse.ie/services/pharmacies-flu-and-covid-vaccines/" rel="canonical"/>
  <meta content="HSE.ie" property="og:site_name"/>
  <meta content="https://www2.hse.ie/services/pharmacies-flu-and-covid-vaccines/" property="og:url"/>
  <meta content="website" property="og:type"/>
  <meta content="Find a pharmacy giving flu and COVID-19 vaccines" property="og:title"/>
  <meta content="Find a pharmacy and contact them to book a Flu or COVID-19 vaccine appointment." property="og:description"/>
  <meta content="https://assets.hse.ie/media/images/HSE_1200x628.max-1200x675.png" property="og:image"/>
  <meta content="summary" name="twitter:card"/>
  <meta content="@hselive" property="twitter:site"/>
  <meta content="Find a pharmacy giving flu and COVID-19

In [41]:
!pip install selenium

   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.7 MB 9.9 MB/s eta 0:00:01
   ---- ----------------------------------- 1.0/9.7 MB 15.4 MB/s eta 0:00:01
   -------- ------------------------------- 2.0/9.7 MB 17.7 MB/s eta 0:00:01
   ----------- ---------------------------- 2.9/9.7 MB 16.6 MB/s eta 0:00:01
   -------------- ------------------------- 3.5/9.7 MB 17.2 MB/s eta 0:00:01
   ------------------ --------------------- 4.5/9.7 MB 17.8 MB/s eta 0:00:01
   -------------------- ------------------- 5.1/9.7 MB 16.2 MB/s eta 0:00:01
   ----------------------- ---------------- 5.7/9.7 MB 16.6 MB/s eta 0:00:01
   --------------------------- ------------ 6.7/9.7 MB 16.5 MB/s eta 0:00:01
   ------------------------------- -------- 7.6/9.7 MB 16.8 MB/s eta 0:00:01
   ----------------------------------- ---- 8.6/9.7 MB 17.8 MB/s eta 0:00:01
   ---------------------------------------  9.5/9.7 MB 17.4 MB/s eta 0:00:01
   -----

In [43]:
#### Was a dynamic page so we need Selenium to handle that - <select> tag and its id or attributes missing or empty, the content is likely generated dynamically.
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Set up Selenium WebDriver
driver = webdriver.Chrome()  # Replace with the path to your ChromeDriver

# Open the URL
url = "https://www2.hse.ie/services/pharmacies-flu-and-covid-vaccines/"
driver.get(url)

try:
    # Wait for the dropdown to load
    dropdown = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "service_area"))
    )

    # Extract all options from the dropdown
    options = dropdown.find_elements(By.TAG_NAME, "option")
    for option in options:
        print(f"Text: {option.text}, Value: {option.get_attribute('value')}")
finally:
    driver.quit()  # Close the browser

Text: Please select, Value: 
Text: Carlow, Value: Carlow
Text: Cavan, Value: Cavan
Text: Clare, Value: Clare
Text: Cork, Value: Cork
Text: Donegal, Value: Donegal
Text: Dublin, Value: Dublin
Text: Dublin 1, Value: Dublin 1
Text: Dublin 10, Value: Dublin 10
Text: Dublin 11, Value: Dublin 11
Text: Dublin 12, Value: Dublin 12
Text: Dublin 13, Value: Dublin 13
Text: Dublin 14, Value: Dublin 14
Text: Dublin 15, Value: Dublin 15
Text: Dublin 16, Value: Dublin 16
Text: Dublin 17, Value: Dublin 17
Text: Dublin 18, Value: Dublin 18
Text: Dublin 2, Value: Dublin 2
Text: Dublin 20, Value: Dublin 20
Text: Dublin 22, Value: Dublin 22
Text: Dublin 24, Value: Dublin 24
Text: Dublin 3, Value: Dublin 3
Text: Dublin 4, Value: Dublin 4
Text: Dublin 5, Value: Dublin 5
Text: Dublin 6, Value: Dublin 6
Text: Dublin 6W, Value: Dublin 6W
Text: Dublin 7, Value: Dublin 7
Text: Dublin 8, Value: Dublin 8
Text: Dublin 9, Value: Dublin 9
Text: Dublin South, Value: Dublin South
Text: Galway, Value: Galway
Text: Kerry

In [3]:
###########Had wrong tags
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv

# Set up the WebDriver
driver = webdriver.Chrome()  # Our browser driver - using chrome
url = "https://www2.hse.ie/services/pharmacies-flu-and-covid-vaccines/"
driver.get(url)

# Output file
output_file = "pharmacy_data_generalized.csv"

try:
    # Open CSV file for writing
    with open(output_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # Write the header
        writer.writerow(["City", "Pharmacy Name", "Description", "Services", "Additional Info"])

        # Wait for the dropdown to load
        dropdown_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "service_area"))
        )
        dropdown = dropdown_element.find_elements(By.TAG_NAME, "option")

        for option in dropdown:
            # Select each city
            option.click()
            city_name = option.text

            # Wait for data to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CLASS_NAME, "hse-grid-row"))
            )

            # Extract pharmacy details
            pharmacies = driver.find_elements(By.CLASS_NAME, "hse-grid-row")
            for pharmacy in pharmacies:
                # Extract pharmacy name
                try:
                    name = pharmacy.find_element(By.TAG_NAME, "h3").text.strip()
                except:
                    name = "NA"

                # Extract description
                try:
                    description = pharmacy.find_element(By.TAG_NAME, "p").text.strip()
                except:
                    description = "NA"

                # Extract services (all links in ordered lists)
                try:
                    services = [link.text.strip() for link in pharmacy.find_elements(By.CSS_SELECTOR, "ol > li > a")]
                except:
                    services = []

                # Extract additional information (unordered lists or other elements)
                try:
                    additional_info = [info.text.strip() for info in pharmacy.find_elements(By.CSS_SELECTOR, "ul > li")]
                except:
                    additional_info = []

                # Write the row
                writer.writerow([city_name, name, description, ", ".join(services), ", ".join(additional_info)])

finally:
    driver.quit() 

In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import csv
import time

# Set up WebDriver
driver = webdriver.Chrome() 
url = "https://www2.hse.ie/services/pharmacies-flu-and-covid-vaccines/"
driver.get(url)

# Output file
output_file = "pharmacy_data_full.csv"

try:
    with open(output_file, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        # Write CSV Header
        writer.writerow([
            "City", "Pharmacy Name", "Address", "Phone Number", "Website", "Email", 
            "Opening Hours", "Flu Vaccines Offered", "COVID-19 Vaccines Offered"
        ])

        # Wait for the dropdown menu to load
        dropdown = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "service_area"))
        )
        options = dropdown.find_elements(By.TAG_NAME, "option")
        
        # Skip the first option (usually a placeholder)
        for option in options[1:]:  
            try:
                city = option.text
                print(f"Processing city: {city}")
                option.click()
                
                # Wait for pharmacy list to load
                time.sleep(2)  # Add a small delay
                WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located((By.CLASS_NAME, "hse-grid-row"))
                )
                pharmacies = driver.find_elements(By.CLASS_NAME, "hse-grid-row")
                
                for pharmacy in pharmacies:
                    try:
                        # Extract pharmacy name
                        name = pharmacy.find_element(By.CSS_SELECTOR, "h3.hse-u-margin-bottom-0").text.strip()
                        
                        # Extract address
                        address = pharmacy.find_element(By.CSS_SELECTOR, "p.hse-u-margin-bottom-2").text.strip()
                        
                        # Extract phone number
                        try:
                            phone = pharmacy.find_element(By.CSS_SELECTOR, "a[href^='tel:']").text.strip()
                        except (NoSuchElementException, Exception):
                            phone = "NA"
                        
                        # Extract website
                        try:
                            website = pharmacy.find_element(By.CSS_SELECTOR, "a[rel='nofollow noopener']").get_attribute("href")
                        except (NoSuchElementException, Exception):
                            website = "NA"
                        
                        # Extract email
                        try:
                            email = pharmacy.find_element(By.CSS_SELECTOR, "a[href^='mailto:']").text.strip()
                        except (NoSuchElementException, Exception):
                            email = "NA"
                        
                        # Extract opening hours
                        try:
                            hours = pharmacy.find_element(By.CSS_SELECTOR, "p[data-v-8f05ba0c]").text.strip()
                        except (NoSuchElementException, Exception):
                            hours = "NA"
                        
                        # Extract flu vaccines offered
                        try:
                            flu_vaccines_elem = pharmacy.find_element(By.XPATH, ".//p[strong[text()='Flu vaccine being offered:']]")
                            flu_vaccines = flu_vaccines_elem.text.strip()
                            flu_list = [li.text for li in pharmacy.find_elements(By.XPATH, ".//p[strong[text()='Flu vaccine being offered:']]/following-sibling::ul/li")]
                            flu_vaccines += "; " + ", ".join(flu_list) if flu_list else flu_vaccines
                        except (NoSuchElementException, Exception):
                            flu_vaccines = "NA"
                        
                        # Extract COVID-19 vaccines offered
                        try:
                            covid_vaccines_elem = pharmacy.find_element(By.XPATH, ".//p[strong[text()='COVID-19 vaccines being offered:']]")
                            covid_vaccines = covid_vaccines_elem.text.strip()
                            covid_list = [li.text for li in pharmacy.find_elements(By.XPATH, ".//p[strong[text()='COVID-19 vaccines being offered:']]/following-sibling::ul/li")]
                            covid_vaccines += "; " + ", ".join(covid_list) if covid_list else covid_vaccines
                        except (NoSuchElementException, Exception):
                            covid_vaccines = "NA"
                        
                        # Write data row to CSV
                        writer.writerow([
                            city, name, address, phone, website, email, hours, flu_vaccines, covid_vaccines
                        ])
                        
                        print(f"Processed pharmacy: {name}")
                    
                    except Exception as pharmacy_error:
                        print(f"Error processing pharmacy: {pharmacy_error}")
                        continue
            
            except Exception as city_error:
                print(f"Error processing city {city}: {city_error}")
                continue
    
    print("Scraping completed successfully!")

except Exception as e:
    print(f"An unexpected error occurred: {e}")

finally:
    driver.quit()

Processing city: Carlow
Error processing pharmacy: Message: no such element: Unable to locate element: {"method":"css selector","selector":"h3.hse-u-margin-bottom-0"}
  (Session info: chrome=131.0.6778.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF721276CB5+28821]
	(No symbol) [0x00007FF7211E3840]
	(No symbol) [0x00007FF72108578A]
	(No symbol) [0x00007FF7210D91BE]
	(No symbol) [0x00007FF7210D94AC]
	(No symbol) [0x00007FF7210CC52C]
	(No symbol) [0x00007FF7210FF33F]
	(No symbol) [0x00007FF7210CC3F6]
	(No symbol) [0x00007FF7210FF510]
	(No symbol) [0x00007FF72111F412]
	(No symbol) [0x00007FF7210FF0A3]
	(No symbol) [0x00007FF7210CA778]
	(No symbol) [0x00007FF7210CB8E1]
	GetHandleVerifier [0x00007FF7215AFCAD+3408013]
	GetHandleVerifier [0x00007FF7215C741F+3504127]
	GetHandleVerifier [0x00007FF7215BB5FD+3455453]
	GetHandleVerifier [0x00007FF72133BDBB